# Исполнение кода в одном потоке, модуль select

Исполнение кода в один поток:
* Подходы для организации "параллельного выполнения кода" в одном потоке
* Использование модуля select
* Неблокирующий ввод/вывод
* Python фрэймворки

В операционной системе существует модуль `select` который позволяет организовать работу с неблокирующим вводом/выводом:

* Модуль `select` используется для организации неблокирующего ввода/вывода
* Существует несколько механизмов опроса файловых дескрипторов
  * select.select(...)
  * select.poll(...)
  * select.epoll(...)
  * select.kqueue(...)
  * ...

##  Неблокирующий ввод/вывод. Обучающий пример

In [ ]:
import socket
import select

sock = socket.socket()
sock.bind(('', 10001))
sock.listen()

# Как обработать запросы для conn1 и conn2 одновременно без потоков?
# Создаем 2 соединения с сокетом
conn1, addr = sock.accept()
conn2, addr = sock.accept()

# Указываем, что режим соединения с сокетом - неблокирующий.
# Это значит, что если данных в сокете не окажется после чтения вызовом recv, то sock.accept() не заблокируется,
# и нам вернет системную ошибку, что пока данных нет.
conn1.setblocking(0)
conn2.setblocking(0)

epoll = select.epoll()
epoll.register(conn1.fileno(), select.EPOLLIN | select.EPOLLOUT)
epoll.register(conn2.fileno(), select.EPOLLIN | select.EPOLLOUT)

conn_map = {
    conn1.fileno(): conn1,
    conn2.fileno(): conn2,
}

# Цикл обработки событий в epoll
while True:
    # Это системный вызов, который возвращает список событий. Эти события содержат
    # файловый дескриптор и событие которое произошло с этим файловым дескриптором.
    events = epoll.poll(1)

    for fileno, event in events:
        if event & select.EPOLLIN:
            data = conn_map[fileno].recv(1024) # Читаем данные из сокета
            print(data.decode('utf8'), end='')
        elif event & select.EPOLLOUT:
            conn_map[fileno].send('ping'.encode('utf8')) # Отправляем данные в сокет

Неблокирующий ввод/вывод:
* Код уже не выглядит слишком простым (хотя в нем нет создания процессов или потоков)
  * Нет обработки закрытия сокетов
  * Отсутствует обработка новых входящих соединений
* Если код будет решать настоящие задачи, то увеличится кол-во операторов if или callback'ов
* Как изменится код, если в обработке запроса появятся вызовы сторонних библиотек?
* Не тратим память на создание процессов
* Нет накладных расходов на создание потоков и их синхронизацию (используются блокировки)
* Нет проблем с GIL
* Как спрятать вызовы select.epoll в функции библиотеки?

Фрэймворки для параллельной работы в один поток:
* Twisted - использует callback api
* Gevent, greenlet, stackless python
* Tornado - использует generators api python3
* Asincio - это сейчас mainstream